<a href="https://colab.research.google.com/github/ddgerrard/MNIST_logistic_regression/blob/master/WeeklyMealPlan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Where to pick up next time (1/18):
# TODO(dgerrard): Make the df the recipe list. Remove a Recipe from the list when it is overwritten.
# TODO(dgerrard): .add_ingredient()
# TODO(dgerrard): Add place to add snacks, etc.
# TODO(dgerrard): Add function to populate dates to WeeklyMeals at instantiation.


In [2]:
from typing import Dict, Optional, Set, Tuple
from enum import Enum

from gspread_dataframe import set_with_dataframe
from google.colab import auth
import gspread
from google.auth import default#autenticating to google
import pandas as pd
import random

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# All Ingredients need to be listed here
class Ingredients(Enum):
  APPLES = ('bag of apples', 'Trader Joes', 'weekly', 'produce')
  AVOCADO = ('avocado', 'Trader Joes', 'weekly', 'produce')
  BACON = ('bacon', 'Lucky', 'weekly', 'meat')
  BANANAS = ('bananas', 'Trader Joes', 'weekly', 'produce')
  BERRIES = ('berries (whatever looks good)', 'Trader Joes', 'weekly', 'produce')
  BLACK_BEANS = ('black beans', 'Trader Joes', 'staple', 'ingredients')
  BROCCOLI = ('broccoli', 'Lucky', 'weekly', 'produce')
  BROWN_RICE = ('brown rice', 'Trader Joes', 'staple', 'grains')
  BROWN_SUGAR = ('brown sugar', 'Lucky', 'staple', 'ingredients')
  BRUSSEL_SPROUTS = ('brussel sprouts', 'Lucky', 'weekly', 'produce')
  CABBAGE = ('cabbage', 'Lucky', 'weekly', 'produce')
  CAPERS = ('capers', 'Trader Joes', 'staple', 'ingredients')
  CARROTS = ('carrots', 'Lucky', 'weekly', 'produce')
  CEASAR_SALAD = ('ceasar salad', 'Lucky', 'weekly', 'produce')
  CHEERIOS = ('joes os - cheerios', 'Trader Joes', 'weekly', 'cereal')
  CHEESE_MONTEREY_JACK = ('monterey jack cheese', 'Trader Joes', 'weekly', 'dairy')
  CHEESE_SHREDDED_MOZARELLA = ('shredded mozarella', 'Trader Joes', 'weekly', 'dairy')
  CHICKEN = ('chicken', 'Costco', 'staple', 'meat')
  CHICKEN_BROTH = ('chicken broth', 'Lucky', 'staple', 'ingredients')
  CHICKEN_NUGGETS = ('chicken nuggets', 'Trader Joes', 'weekly', 'frozen')
  CHICKEN_THIGHS = ('chicken thighs', 'Lucky', 'weekly', 'meat')
  CILANTRO = ('cilantro', 'Lucky', 'weekly', 'produce')
  COCONUT_MILK = ('coconut milk', 'Trader Joes', 'weekly', 'ingredients')
  COSTCO_BAGUETTE = ('costco baguette', 'Costco', 'weekly', 'grains')
  COTIJA = ('cotija cheese', 'Trader Joes', 'staple', 'dairy')
  CUCUMBERS = ('pack of cucumbers', 'Trader Joes', 'weekly', 'produce')
  CUMIN = ('cumin', 'Lucky', 'staple', 'ingredients')
  CUMIN_SEEDS = ('cumin seeds', 'Lucky', 'staple', 'ingredients')
  CURRY_POWDER = ('curry powder', 'Lucky', 'staple', 'ingredients')
  DAVES_KILLER_BREAD = ('daves killer bread', 'Lucky', 'weekly', 'grains')
  DICED_GREEN_CHILES_7OZ = ('diced green chiles 7oz', 'Trader Joes', 'weekly', 'ingredients')
  DIJON_MUSTARD = ('dijon mustard', 'Lucky', 'staple', 'ingredients')
  DRIED_BASIL = ('dried basil', 'Lucky', 'staple', 'ingredients')
  DRIED_MANGO = ('dried mango', 'Trader Joes', 'weekly', 'snack')
  DRIED_OREGANO = ('dried oregano', 'Lucky', 'staple', 'ingredients')
  DRIED_PARSLEY = ('dried parsley', 'Lucky', 'staple', 'ingredients')
  DRIED_THYME = ('dried thyme', 'Lucky', 'staple', 'ingredients')
  DRY_CHICKPEAS = ('dry chickpeas', 'Costco', 'staple', 'grains')
  EGGS = ('eggs', 'Trader Joes', 'weekly', 'dairy')
  FETA = ('feta', 'Trader Joes', 'weekly', 'dairy')
  FLANK_STEAK = ('flank steak', 'Trader Joes', 'weekly', 'meat')
  FLATBREAD = ('flatbread', 'Trader Joes', 'weekly', 'grains')
  FLOUR_TORTILLAS = ('flour tortillas (del comal)', 'Trader Joes', 'weekly', 'grains')
  FRESH_BASIL = ('fresh basil', 'Lucky', 'weekly', 'produce')
  GARLIC = ('garlic', 'Lucky', 'staple', 'produce')
  GARLIC_POWDER = ('garlic powder', 'Lucky', 'staple', 'ingredients')
  GOCHUJANG_SAUCE = ('gochujang suace', 'Lucky', 'staple', 'ingredients')
  GREEN_BEANS = ('green beans', 'Trader Joes', 'weekly', 'produce')
  HEIRLOOM_TOMATOES = ('heirloom tomatoes', 'Lucky', 'weekly', 'produce')
  ITALIAN_SAUSAGE = ('aidells italian style sausage', 'Lucky', 'weekly', 'meat')
  KALE = ('kale', 'Lucky', 'weekly', 'produce')
  KETCHUP = ('ketchup', 'Lucky', 'staple', 'ingredients')
  KOSHER_SALT = ('kosher salt', 'Lucky', 'staple', 'ingredients')
  LEMON = ('lemon', 'Trader Joes', 'weekly', 'produce')
  MAC_AND_CHEESE = ('mac & cheese', 'Trader Joes', 'weekly', 'grains')
  MAYONAISE = ('mayonaise', 'Lucky', 'staple', 'ingredients')
  MILK_HALF_GAL_2P = ('half gallon 2% milk', 'Trader Joes', 'weekly', 'dairy')
  MILK_GAL_WHOLE = ('gallon whole milk', 'Trader Joes', 'weekly', 'dairy')
  OATMEAL = ('oatmeal', 'Trader Joes', 'staple', 'grains')
  OLIVE_OIL = ('olive oil', 'Trader Joes', 'staple', 'ingredients')
  ONION_POWDER = ('onion powder', 'Lucky', 'staple', 'ingredients')
  ONION_RED = ('red onion', 'Trader Joes', 'weekly', 'produce')
  ONION_YELLOW = ('onion - yellow', 'Trader Joes', 'weekly', 'produce')
  PANCAKE_MIX = ('pancake mix', 'Costco', 'staple', 'grains')
  PARMESAN = ('parmesan', 'Trader Joes', 'staple', 'ingredients')
  PASTA = ('pasta', 'Trader Joes', 'weekly', 'grains')
  PASTA_SAUCE = ('pasta sauce', 'Trader Joes', 'weekly', 'ingredients')
  PEANUT_BUTTER_PUFFINS = ('peanut butter puffins', 'Trader Joes', 'weekly', 'cereal')
  PEANUTS_UNSALTED = ('unsalted peanuts', 'Trader Joes', 'staple', 'ingredients')
  PEAS = ('peas', 'Trader Joes', 'weekly', 'produce')
  PEPPERONIS = ('pepperonis', 'Lucky', 'staple', 'meat')
  PESTO = ('pesto', 'Costco', 'staple', 'ingredients')
  PITA_CHIPS = ('pita chips (blue bag)', 'Trader Joes', 'weekly', 'snack')
  PIZZA_DOUGH = ('pizza dough', 'Trader Joes', 'weekly', 'ingredients')
  PIZZA_SAUCE = ('pizza sauce', 'Trader Joes', 'weekly', 'ingredients')
  PORK_LOIN = ('pork loin', 'Trader Joes', 'weekly', 'meat')
  POTATOES_SMALL_GOLDEN = ('small golden potatoes', 'Trader Joes', 'weekly', 'produce')
  RAMEN_NOODLES = ('ramen noodles - chicken', 'Lucky', 'weekly', 'ingredients')
  RED_BELL_PEPPER = ('red bell pepper', 'Trader Joes', 'weekly', 'produce')
  RED_POTATOES = ('red potatoes', 'Lucky', 'weekly', 'produce')
  RICE_COCONUT = ('coconut rice', 'Lucky', 'staple', 'grains')
  RICE_VINEGAR = ('rice vinegar', 'Trader Joes', 'staple', 'ingredients')
  ROTISSERIE_CHICKEN = ('rotisserie chicken', 'Costco', 'weekly', 'meat')
  SALMON = ('salmon', 'Lucky', 'weekly', 'meat')
  SALSA = ('salsa', 'Lucky', 'weekly', 'ingredients')
  SALTED_BUTTER = ('salted butter', 'Trader Joes', 'staple', 'dairy')
  SEAWEED = ('seaweed', 'Costco', 'staple', 'snacks')
  SESAME_OIL = ('sesame oil', 'Trader Joes', 'staple', 'ingredients')
  SHREDDED_CHEESE = ('shredded mexican cheese', 'Trader Joes', 'staple', 'dairy')
  SOUR_CREAM = ('sour cream', 'Trader Joes', 'weekly', 'dairy')
  SOY_SAUCE = ('soy sauce', 'Lucky', 'staple', 'ingredients')
  SPINACH_BAG = ('bag of spinach', 'Trader Joes', 'weekly', 'produce')
  SUGAR = ('sugar', 'Lucky', 'staple', 'ingredients')
  SYRUP = ('syrup', 'Lucky', 'staple', 'ingredients')
  TAHINI = ('tahini', 'Trader Joes', 'staple', 'ingredients')
  TARAGON = ('taragon', 'Lucky', 'weekly', 'produce')
  TERIYAKI_SAUCE = ('teriyaki sauce', 'Lucky', 'staple', 'ingredients')
  THYME_SPRIGS = ('thyme sprigs', 'Lucky', 'weekly', 'produce')
  TOMA = ('toma', 'Costco', 'weekly', 'dairy')
  TOMATOES = ('tomato', 'Trader Joes', 'weekly', 'produce')
  TOMATO_SOUP = ('tomato soup', 'Trader Joes', 'weekly', 'ingredients')
  TRAIL_MIX_RAINBOW = ('trail mix - rainbow', 'Trader Joes', 'weekly', 'snack')
  VEGETABLE_OIL = ('vegetable oil', 'Lucky', 'staple', 'ingredients')
  WHIPPED_CREAM = ('whipped cream', 'Lucky', 'staple', 'dairy')
  WHITE_BEANS = ('white beans - canelli', 'Trader Joes', 'weekly', 'ingredients')
  WHITE_BALSAMIC = ('white balsamic vinegar', 'Lucky', 'staple', 'ingredients')
  WHITE_VINEGAR = ('white vinegar', 'Lucky', 'staple', 'ingredients')
  WORCESTERSHIRE_SAUCE = ('worcestershire sauce', 'Lucky', 'staple', 'ingredients')
  Z_BAR = ('z-bar', 'Trader Joes', 'weekly', 'snack')
  ZUCCHINI = ('zucchini', 'Lucky', 'weekly', 'produce')

class Meas(Enum):
  BAG = 'bag(s)'
  BOX = 'box(es)'
  BUNCH = 'bunch'
  CAN = 'can(s)'
  CHEAPEST = 'cheapest'
  COUNT = ''
  CLOVES = 'cloves'
  CUP = 'cup(s)'
  JUG = 'jug'
  LB = 'lb'
  LBS = 'lb(s)'
  PACK = 'pack'
  TBSP = 'Tablespoon'
  TSP = 'teaspoon'
  TYPES = 'types'

class MealTags(Enum):
  WEEKDAY_LUNCH = 'weekday lunch'
  WEEKEND_LUNCH = 'weekend lunch'
  EASY_DINNER = 'easy dinner'
  DINNER = 'dinner'


In [3]:
class Recipe():

  def __init__(self, name: str, meal_tags: Set[MealTags], ingredients: Dict[Ingredients, Tuple]):
    self.name = name
    self.meal_tags = meal_tags
    self.ingredients = ingredients



In [4]:
# done
quick_fire_curry_chicken = Recipe('Curry Chicken', [MealTags.DINNER], {Ingredients.OLIVE_OIL:(3, Meas.TBSP),
                                                               Ingredients.ONION_YELLOW:(1, Meas.COUNT),
                                                               Ingredients.CUMIN_SEEDS:(0.5, Meas.TSP),
                                                               Ingredients.CURRY_POWDER:(2, Meas.TSP),
                                                               Ingredients.KOSHER_SALT:(0.5, Meas.TSP),
                                                               Ingredients.COCONUT_MILK:(1, Meas.CAN),
                                                               Ingredients.ROTISSERIE_CHICKEN:(2, Meas.CUP),
                                                               Ingredients.RICE_COCONUT:(1, Meas.COUNT),
                                                               Ingredients.CILANTRO:(0.25, Meas.CUP)})

# done
bibimbaap = Recipe('Bibimbaap', [MealTags.DINNER], {Ingredients.FLANK_STEAK:(1, Meas.CHEAPEST),
                                 Ingredients.EGGS:(6, Meas.COUNT),
                                 Ingredients.SPINACH_BAG:(1, Meas.BAG),
                                 Ingredients.CARROTS:(2, Meas.COUNT),
                                 Ingredients.WORCESTERSHIRE_SAUCE:(1, Meas.COUNT),
                                 Ingredients.KOSHER_SALT:(1, Meas.COUNT),
                                 Ingredients.GOCHUJANG_SAUCE:(4, Meas.TBSP),
                                 Ingredients.SOY_SAUCE:(1, Meas.TBSP),
                                 Ingredients.BROWN_SUGAR:(2, Meas.TBSP),
                                 Ingredients.RICE_VINEGAR:(2, Meas.TSP),
                                 Ingredients.SESAME_OIL:(2, Meas.TBSP),
                                 Ingredients.SEAWEED:(1, Meas.COUNT)})

# done
mac_and_cheese = Recipe('Mac & Cheese', [MealTags.EASY_DINNER], {Ingredients.MAC_AND_CHEESE:(2, Meas.BOX)})

hummus_and_veggies = Recipe('Hummus & Veggies', [MealTags.EASY_DINNER], {Ingredients.FLATBREAD:(1, Meas.BAG)})

# done
grilled_cheese_sandwich = Recipe('Grilled Cheese & Tomato Soup', [MealTags.EASY_DINNER], {Ingredients.DAVES_KILLER_BREAD:(1, Meas.BAG),
                                                                  Ingredients.CHEESE_MONTEREY_JACK:(1, Meas.PACK),
                                                                  Ingredients.TOMATO_SOUP:(1, Meas.BOX)})

# done
roasted_veggies_and_sausage = Recipe('Roasted Veggies & Sausage', [MealTags.DINNER], {Ingredients.ITALIAN_SAUSAGE:(1, Meas.PACK),
                                                                   Ingredients.CARROTS:(2, Meas.COUNT),
                                                                   Ingredients.RED_POTATOES:(2, Meas.COUNT),
                                                                   Ingredients.ZUCCHINI:(1, Meas.COUNT),
                                                                   Ingredients.RED_BELL_PEPPER:(1, Meas.COUNT),
                                                                   Ingredients.BROCCOLI:(1, Meas.COUNT),
                                                                   Ingredients.DRIED_BASIL:(0.5, Meas.TBSP),
                                                                   Ingredients.DRIED_OREGANO:(0.5, Meas.TBSP),
                                                                   Ingredients.DRIED_PARSLEY:(0.5, Meas.TBSP),
                                                                   Ingredients.GARLIC_POWDER:(0.5, Meas.TBSP),
                                                                   Ingredients.ONION_POWDER:(0.5, Meas.TSP),
                                                                   Ingredients.DRIED_THYME:(0.5, Meas.TSP),
                                                                   Ingredients.PARMESAN:(0.33, Meas.CUP)})

# done
chicken_tacos = Recipe('Tacos & Quesadillas', [MealTags.DINNER], {Ingredients.FLOUR_TORTILLAS:(1, Meas.PACK),
                         Ingredients.AVOCADO:(2, Meas.COUNT),
                         Ingredients.CHICKEN:(1, Meas.CUP),
                         Ingredients.SHREDDED_CHEESE:(0.5, Meas.CUP),
                         Ingredients.COTIJA:(0.25, Meas.CUP),
                         Ingredients.BLACK_BEANS:(1, Meas.CAN),
                         Ingredients.TOMATOES:(3, Meas.COUNT),
                         Ingredients.SALSA:(1, Meas.CUP),
                         Ingredients.RED_BELL_PEPPER:(1, Meas.COUNT),
                         Ingredients.ONION_YELLOW:(1, Meas.COUNT)})

# done
thai_cabbage_salad = Recipe('Thai Cabbage Salad', [MealTags.WEEKDAY_LUNCH], {Ingredients.CHICKEN:(2, Meas.CUP),
                                                   Ingredients.CABBAGE:(1, Meas.COUNT),
                                                   Ingredients.RED_BELL_PEPPER:(1, Meas.COUNT),
                                                   Ingredients.CARROTS:(1, Meas.COUNT),
                                                   Ingredients.CILANTRO:(1, Meas.COUNT),
                                                   Ingredients.VEGETABLE_OIL:(3, Meas.TBSP),
                                                   Ingredients.LEMON:(1, Meas.COUNT),
                                                   Ingredients.SUGAR:(1, Meas.TBSP),
                                                   Ingredients.TERIYAKI_SAUCE:(1, Meas.TBSP),
                                                   Ingredients.WHITE_VINEGAR:(1, Meas.TBSP),
                                                   Ingredients.PEANUTS_UNSALTED:(0.5, Meas.CUP),
                                                   Ingredients.RAMEN_NOODLES:(1, Meas.PACK)})

# done
# makes lots of leftovers
white_bean_chili = Recipe('White Bean Chili', [MealTags.DINNER], {Ingredients.ONION_YELLOW:(1, Meas.COUNT),
                                               Ingredients.CHICKEN_BROTH:(6, Meas.CUP),
                                               Ingredients.CHICKEN:(3, Meas.CUP),
                                               Ingredients.WHITE_BEANS:(3, Meas.CAN),
                                               Ingredients.CHEESE_MONTEREY_JACK:(3, Meas.CUP),
                                               Ingredients.DICED_GREEN_CHILES_7OZ:(1, Meas.CAN),
                                               Ingredients.SOUR_CREAM:(1, Meas.CUP)})


pork_loin_and_roasted_potatoes = Recipe('Pork Loin & Roasted Potatoes', [MealTags.DINNER], {Ingredients.PORK_LOIN:(1, Meas.COUNT),
                                                                         Ingredients.POTATOES_SMALL_GOLDEN:(1, Meas.BAG)})

weekly_snacks = Recipe('Snacks', [], {Ingredients.Z_BAR:(6, Meas.COUNT),
                                  Ingredients.TRAIL_MIX_RAINBOW:(1, Meas.BAG),
                                  Ingredients.DRIED_MANGO:(1, Meas.BAG),
                                  Ingredients.PITA_CHIPS:(1, Meas.BAG)})

weekly_staples = Recipe('Weekly Staples', [], {Ingredients.CHEERIOS:(1, Meas.BOX),
                                           Ingredients.PEANUT_BUTTER_PUFFINS:(1, Meas.BOX),
                                           Ingredients.MILK_HALF_GAL_2P:(1, Meas.JUG),
                                           Ingredients.MILK_GAL_WHOLE:(1, Meas.JUG),
                                           Ingredients.APPLES:(1, Meas.BAG),
                                           Ingredients.CUCUMBERS:(1, Meas.PACK),
                                           Ingredients.PEAS:(1, Meas.BAG),
                                           Ingredients.BANANAS:(6, Meas.COUNT),
                                           Ingredients.BERRIES:(1, Meas.COUNT),
                                           Ingredients.OATMEAL:(1, Meas.PACK)})

frozen_staples = Recipe('Frozen Staples', [], {Ingredients.CHICKEN_NUGGETS:(1, Meas.BOX)})

# done
chicken_nuggets_and_broccoli = Recipe('Chicken Nuggets & Broccoli', [MealTags.WEEKEND_LUNCH], {Ingredients.CHICKEN_NUGGETS:(1, Meas.BOX),
                                                        Ingredients.BROCCOLI:(1, Meas.COUNT),
                                                        Ingredients.KETCHUP:(1, Meas.COUNT)})

# done
trader_joes_pizza = Recipe('Trader Joes Pizza', [MealTags.WEEKEND_LUNCH], {Ingredients.PIZZA_DOUGH:(1, Meas.COUNT),
                                                        Ingredients.CHEESE_SHREDDED_MOZARELLA:(1, Meas.BAG),
                                                        Ingredients.PIZZA_SAUCE:(1, Meas.CAN),
                                                        Ingredients.PEPPERONIS:(1, Meas.BAG)})

# done
breakfast_for_dinner = Recipe('Breakfast for Dinner', [MealTags.DINNER], {Ingredients.EGGS:(6, Meas.COUNT),
                                                       Ingredients.BERRIES:(1, Meas.COUNT),
                                                       Ingredients.PANCAKE_MIX:(1, Meas.COUNT),
                                                       Ingredients.BACON:(10, Meas.COUNT),
                                                       Ingredients.SYRUP:(1, Meas.COUNT),
                                                       Ingredients.SALTED_BUTTER:(1, Meas.COUNT),
                                                       Ingredients.WHIPPED_CREAM:(1, Meas.COUNT)})

# done
stewy_chickpeas = Recipe('Stewy Chickpeas', [MealTags.WEEKDAY_LUNCH], {Ingredients.ONION_RED:(1, Meas.COUNT),
                                             Ingredients.ONION_YELLOW:(0.5, Meas.COUNT),
                                             Ingredients.DRY_CHICKPEAS:(1, Meas.LB),
                                             Ingredients.KOSHER_SALT:(3, Meas.TBSP),
                                             Ingredients.THYME_SPRIGS:(3, Meas.COUNT),
                                             Ingredients.OLIVE_OIL:(0.5, Meas.CUP),
                                             Ingredients.GARLIC:(6, Meas.CLOVES),
                                             Ingredients.HEIRLOOM_TOMATOES:(4, Meas.COUNT),
                                             Ingredients.WHITE_BALSAMIC:(3, Meas.TBSP),
                                             Ingredients.FRESH_BASIL:(1, Meas.BUNCH),
                                             Ingredients.CAPERS:(1, Meas.TBSP),
                                             Ingredients.FETA:(0.5, Meas.CUP)
                                             })


lemon_tahini_chicken = Recipe('Lemon Tahini Chicken', [MealTags.DINNER], {Ingredients.TAHINI:(0.25, Meas.CUP),
                                                       Ingredients.LEMON:(2, Meas.COUNT),
                                                       Ingredients.GARLIC:(3, Meas.CLOVES),
                                                       Ingredients.CUMIN:(0.25, Meas.TSP),
                                                       Ingredients.KOSHER_SALT:(0.5, Meas.TSP),
                                                       Ingredients.GREEN_BEANS:(1, Meas.CUP),
                                                       Ingredients.BROCCOLI:(1, Meas.COUNT),
                                                       Ingredients.CHICKEN:(1, Meas.COUNT),
                                                       Ingredients.OLIVE_OIL:(2, Meas.TBSP),
                                                       Ingredients.ONION_RED:(0.5, Meas.COUNT),
                                                       Ingredients.KALE:(4, Meas.CUP),
                                                       Ingredients.BROWN_RICE:(2, Meas.CUP),
                                                       Ingredients.CILANTRO:(0.25, Meas.CUP)})

pasta = Recipe('Pasta', [MealTags.EASY_DINNER], {Ingredients.PASTA:(2, Meas.TYPES),
                                                 Ingredients.PARMESAN:(1, Meas.CUP),
                                                 Ingredients.PASTA_SAUCE:(1, Meas.CAN),
                                                 Ingredients.PESTO:(1, Meas.CAN),
                                                 Ingredients.TOMATOES:(3, Meas.COUNT)})

chicken_thighs_and_potatoes = Recipe('Chicken Thighs & Potatoes', [MealTags.DINNER], {Ingredients.CHICKEN_THIGHS:(4, Meas.COUNT),
                                                                                      Ingredients.POTATOES_SMALL_GOLDEN:(1, Meas.BAG),
                                                                                      Ingredients.ONION_YELLOW:(1, Meas.COUNT),
                                                                                      Ingredients.OLIVE_OIL:(2, Meas.TBSP),
                                                                                      Ingredients.MAYONAISE:(0.25, Meas.CUP),
                                                                                      Ingredients.DIJON_MUSTARD:(2, Meas.TBSP),
                                                                                      Ingredients.TARAGON:(1, Meas.BUNCH)})

salmon_and_brussel_sprouts = Recipe('Salmon & Brussel Sprouts', [MealTags.DINNER], {Ingredients.SALMON:(1.25, Meas.LBS),
                                                                                    Ingredients.BRUSSEL_SPROUTS:(1, Meas.LB),
                                                                                    Ingredients.CEASAR_SALAD:(1, Meas.BAG),
                                                                                    Ingredients.LEMON:(2, Meas.COUNT)})

costco_sandwiches = Recipe('Costco Sandwiches', [MealTags.WEEKDAY_LUNCH], {Ingredients.ROTISSERIE_CHICKEN:(1, Meas.LBS),
                                                                           Ingredients.COSTCO_BAGUETTE:(1, Meas.BAG),
                                                                           Ingredients.TOMA:(1, Meas.COUNT)})

In [5]:
class AllRecipes():
  def __init__(self):
    self.recipe_set = [quick_fire_curry_chicken,
      bibimbaap,
      mac_and_cheese,
      hummus_and_veggies,
      grilled_cheese_sandwich,
      roasted_veggies_and_sausage,
      chicken_tacos,
      thai_cabbage_salad,
      white_bean_chili,
      pork_loin_and_roasted_potatoes,
      frozen_staples,
      chicken_nuggets_and_broccoli,
      breakfast_for_dinner,
      stewy_chickpeas,
      lemon_tahini_chicken,
      trader_joes_pizza,
      pasta,
      chicken_thighs_and_potatoes,
      salmon_and_brussel_sprouts,
      costco_sandwiches]

  def recipes_with_tag(self, tag: MealTags, number: Optional[int]=None):
    recipe_tag_list = []
    for recipe in self.recipe_set:
      if tag in recipe.meal_tags:
        recipe_tag_list.append(recipe)
    if number is not None:
      return random.sample(recipe_tag_list, number)
    return recipe_tag_list



In [22]:
class WeeklyMeals():

  def __init__(self):
    self.worksheet = gc.open('Meal Planning').sheet1
    self.all_recipes = AllRecipes()
    self.date = '1'
    self.recipes = []
    self.list_of_groceries = {}
    self.store_lists = {}
    self.weekly_meals_df = pd.DataFrame({'Day':['Date','Lunch','Dinner'], 'Sun':['', '', ''], 'Mon':['', '', ''], 'Tues': ['', '', ''], 'Wed': ['', '', ''], 'Thurs': ['', '', ''], 'Fri': ['','', ''], 'Sat':['', '', '']}).set_index('Day')
    # self.weekly_meals_df.set_index('Day')
  
  def generate_a_random_meal_plan(self):
    lunches = ('Lunch', ('Wed', 'Thurs'), self.all_recipes.recipes_with_tag(MealTags.WEEKDAY_LUNCH, 2))
    weekend_lunches = ('Lunch', ('Sat', 'Sun'), self.all_recipes.recipes_with_tag(MealTags.WEEKEND_LUNCH, 2))
    dinners = ('Dinner', ('Mon', 'Tues', 'Thurs'), self.all_recipes.recipes_with_tag(MealTags.DINNER, 3))
    easy_dinners = ('Dinner', ('Wed', 'Fri'), self.all_recipes.recipes_with_tag(MealTags.EASY_DINNER, 2))
    all_meal_categories = [lunches, weekend_lunches, dinners, easy_dinners]
    for meal_time, days, recipe_list in all_meal_categories:
      for day, recipe in zip(days, recipe_list):
        self.add_recipe(recipe, day, '1/1', meal_time)

  def add_recipe(self, recipe: Recipe, day: str, date: str, meal: str):
    self.weekly_meals_df[day].loc['Date'] = date
    self.weekly_meals_df[day].loc[meal] = recipe.name
    self.recipes.append(recipe)

  def make_list_of_groceries(self):
    self.list_of_groceries = {}
    for recipe in self.recipes:
      for ingredient, (count, measurement) in recipe.ingredients.items():
        # print(ingredient)
        # print(count)
        # print(measurement)
        if ingredient in self.list_of_groceries:
          self.list_of_groceries[ingredient] = (self.list_of_groceries[ingredient][0] + count, measurement)
          continue
        self.list_of_groceries[ingredient] = (count, measurement)

  def make_store_lists(self):
    list_of_groceries = self.list_of_groceries
    for ingredient, ingredient_info in list_of_groceries.items():
      # print(ingredient)
      # print(type(ingredient))
      # print(ingredient_info)
      # print(ingredient.value)
      i = ingredient.value[0]
      store = ingredient.value[1]
      freq = ingredient.value[2]
      list_name = store + ' ' + freq
      # print(ingredient)
      # if ingredient.value[2] == 'weekly':
      if list_name in self.store_lists:
          self.store_lists[list_name] = pd.concat([self.store_lists[list_name], pd.DataFrame({'department':[ingredient.value[3]], 'item':[i], 'number':[ingredient_info[0]], 'measure':[ingredient_info[1].value]})])
          # print((ingredient.value[3], i, ingredient_info[0], ingredient_info[1].value))
          # print('\n')
          continue
      self.store_lists[list_name] = pd.DataFrame({'department':[ingredient.value[3]], 'item':[i], 'number':[ingredient_info[0]], 'measure':[ingredient_info[1].value]})
      # print((ingredient.value[3], i, ingredient_info[0], ingredient_info[1].value))
      # print('\n')
    for store_list_name, store_list_df in self.store_lists.items():
      # self.store_lists = store_list.set_index(['department', 'item']).groupby(level=[0,1]).max().sort_index(ascending=False)
      self.store_lists[store_list_name] = store_list_df.set_index(['department', 'item']).groupby(level=[0,1]).max().sort_index(ascending=False)

  def write_meal_schedule_to_sheet(self):
    set_with_dataframe(self.worksheet, self.weekly_meals_df, include_index=True)

  def write_lists_to_sheet(self):
    set_with_dataframe(self.worksheet, pd.DataFrame({'0':['Trader Joes']}), row=6, col=1, include_column_header=False)
    set_with_dataframe(self.worksheet, pd.DataFrame({'0':['Lucky']}), row=6, col=6, include_column_header=False)
    set_with_dataframe(self.worksheet, pd.DataFrame({'0':['Costco']}), row=6, col=11, include_column_header=False)
    set_with_dataframe(self.worksheet, self.store_lists['Trader Joes weekly'], row=7, col=1, include_index=True, include_column_header=False) 
    set_with_dataframe(self.worksheet, self.store_lists['Lucky weekly'], row=7, col=6, include_index=True, include_column_header=False)
    if 'Costco weekly' in self.store_lists.keys():
      set_with_dataframe(self.worksheet, self.store_lists['Costco weekly'], row=7, col=11, include_index=True, include_column_header=False)
    if 'Costco weekly' in self.store_lists.keys():
      row = 8 + max([len(self.store_lists['Trader Joes weekly']), len(self.store_lists['Lucky weekly']), len(self.store_lists['Costco weekly'])])
    else:
      row = 8 + max([len(self.store_lists['Trader Joes weekly']), len(self.store_lists['Lucky weekly'])])
    set_with_dataframe(self.worksheet, pd.DataFrame({'0':['Staples']}), row=row, col=1, include_column_header=False)
    set_with_dataframe(self.worksheet, self.store_lists['Trader Joes staple'], row=row+1, col=1, include_index=True, include_column_header=False) 
    set_with_dataframe(self.worksheet, self.store_lists['Lucky staple'], row=row+1, col=6, include_index=True, include_column_header=False)
    set_with_dataframe(self.worksheet, self.store_lists['Costco staple'], row=row+1, col=11, include_index=True, include_column_header=False)



In [9]:
sl = WeeklyMeals()
sl.generate_a_random_meal_plan()
sl.weekly_meals_df

,Sun,Mon,Tues,Wed,Thurs,Fri,Sat
Day,,,,,,,
Date,1/1,1/1,1/1,1/1,1/1,1/1,1/1
Lunch,Trader Joes Pizza,,,Thai Cabbage Salad,Stewy Chickpeas,,Chicken Nuggets & Broccoli
Dinner,,Salmon & Brussel Sprouts,Chicken Thighs & Potatoes,Mac & Cheese,Bibimbaap,Pasta,


In [23]:
sl = WeeklyMeals()
sl.add_recipe(stewy_chickpeas, 'Wed', '1/18', 'Lunch')
sl.add_recipe(salmon_and_brussel_sprouts, 'Wed', '1/18', 'Dinner')
sl.add_recipe(lemon_tahini_chicken, 'Thurs', '1/19', 'Dinner')
sl.add_recipe(thai_cabbage_salad, 'Fri', '1/20', 'Lunch')
sl.add_recipe(pasta, 'Fri', '1/20', 'Dinner')
sl.add_recipe(costco_sandwiches, 'Sat', '1/21', 'Lunch')
sl.add_recipe(pork_loin_and_roasted_potatoes, 'Sat', '1/21', 'Dinner')
sl.add_recipe(chicken_nuggets_and_broccoli, 'Sun', '1/22', 'Lunch')
sl.add_recipe(bibimbaap, 'Sun', '1/22', 'Dinner')
sl.add_recipe(quick_fire_curry_chicken, 'Mon', '1/23', 'Dinner')
sl.add_recipe(weekly_snacks, 'Mon', '1/23', 'Lunch')
sl.add_recipe(weekly_staples, 'Tues', '1/24', 'Lunch')

# sl.add_ingredient('baby kale')


sl.make_list_of_groceries()
sl.make_store_lists()
sl.store_lists
sl.weekly_meals_df

,Sun,Mon,Tues,Wed,Thurs,Fri,Sat
Day,,,,,,,
Date,1/22,1/23,1/24,1/18,1/19,1/20,1/21
Lunch,Chicken Nuggets & Broccoli,Snacks,Weekly Staples,Stewy Chickpeas,,Thai Cabbage Salad,Costco Sandwiches
Dinner,Bibimbaap,Curry Chicken,,Salmon & Brussel Sprouts,Lemon Tahini Chicken,Pasta,Pork Loin & Roasted Potatoes


In [20]:
sl.store_lists

{}

In [11]:
sl.make_store_lists()
sl.store_lists

{'Trader Joes weekly':                                     number  measure
 department  item                                   
 produce     tomato                     6.0         
             red onion                  3.0         
             red bell pepper            1.0         
             onion - yellow             3.0         
             lemon                      5.0         
             green beans                2.0   cup(s)
 ingredients white beans - canelli      3.0   can(s)
             tomato soup                1.0  box(es)
             pizza sauce                1.0   can(s)
             pizza dough                1.0         
             pasta sauce                2.0   can(s)
             diced green chiles 7oz     1.0   can(s)
             coconut milk               1.0   can(s)
 grains      pasta                      4.0    types
 frozen      chicken nuggets            1.0  box(es)
 dairy       sour cream                 1.0   cup(s)
             shredded mo

In [12]:
sl.store_lists['Trader Joes weekly']

number  measure
department  item                                   
produce     tomato                     6.0         
            red onion                  3.0         
            red bell pepper            1.0         
            onion - yellow             3.0         
            lemon                      5.0         
            green beans                2.0   cup(s)
ingredients white beans - canelli      3.0   can(s)
            tomato soup                1.0  box(es)
            pizza sauce                1.0   can(s)
            pizza dough                1.0         
            pasta sauce                2.0   can(s)
            diced green chiles 7oz     1.0   can(s)
            coconut milk               1.0   can(s)
grains      pasta                      4.0    types
frozen      chicken nuggets            1.0  box(es)
dairy       sour cream                 1.0   cup(s)
            shredded mozarella         1.0   bag(s)
            monterey jack cheese       4.0     pack
            feta                       1.0   cup(s)

In [24]:
sl.write_meal_schedule_to_sheet()

In [25]:
sl.write_lists_to_sheet()

In [15]:
sl.weekly_meals_df

,Sun,Mon,Tues,Wed,Thurs,Fri,Sat
Day,,,,,,,
Date,1/1,1/1,1/1,1/18,1/19,1/1,1/1
Lunch,Trader Joes Pizza,,,Stewy Chickpeas,Thai Cabbage Salad,,Chicken Nuggets & Broccoli
Dinner,,White Bean Chili,Lemon Tahini Chicken,Lemon Tahini Chicken,Pasta,Pasta,


In [16]:
set_with_dataframe(sl.worksheet, sl.store_lists['Trader Joes weekly'], row=6, col=1, include_index=True, include_column_header=False) 

In [17]:
sl.recipes[0].ingredients

{<Ingredients.ONION_RED: ('red onion', 'Trader Joes', 'weekly', 'produce')>: (1,
  <Meas.COUNT: ''>),
 <Ingredients.ONION_YELLOW: ('onion - yellow', 'Trader Joes', 'weekly', 'produce')>: (0.5,
  <Meas.COUNT: ''>),
 <Ingredients.DRY_CHICKPEAS: ('dry chickpeas', 'Costco', 'staple', 'grains')>: (1,
  <Meas.LB: 'lb'>),
 <Ingredients.KOSHER_SALT: ('kosher salt', 'Lucky', 'staple', 'ingredients')>: (3,
  <Meas.TBSP: 'Tablespoon'>),
 <Ingredients.THYME_SPRIGS: ('thyme sprigs', 'Lucky', 'weekly', 'produce')>: (3,
  <Meas.COUNT: ''>),
 <Ingredients.OLIVE_OIL: ('olive oil', 'Trader Joes', 'staple', 'ingredients')>: (0.5,
  <Meas.CUP: 'cup(s)'>),
 <Ingredients.GARLIC: ('garlic', 'Lucky', 'staple', 'produce')>: (6,
  <Meas.CLOVES: 'cloves'>),
 <Ingredients.HEIRLOOM_TOMATOES: ('heirloom tomatoes', 'Lucky', 'weekly', 'produce')>: (4,
  <Meas.COUNT: ''>),
 <Ingredients.WHITE_BALSAMIC: ('white balsamic vinegar', 'Lucky', 'staple', 'ingredients')>: (3,
  <Meas.TBSP: 'Tablespoon'>),
 <Ingredients.FRESH_

In [18]:
sl.list_of_groceries

{<Ingredients.ONION_RED: ('red onion', 'Trader Joes', 'weekly', 'produce')>: (3.0,
  <Meas.COUNT: ''>),
 <Ingredients.ONION_YELLOW: ('onion - yellow', 'Trader Joes', 'weekly', 'produce')>: (3.0,
  <Meas.COUNT: ''>),
 <Ingredients.DRY_CHICKPEAS: ('dry chickpeas', 'Costco', 'staple', 'grains')>: (2,
  <Meas.LB: 'lb'>),
 <Ingredients.KOSHER_SALT: ('kosher salt', 'Lucky', 'staple', 'ingredients')>: (7.5,
  <Meas.TSP: 'teaspoon'>),
 <Ingredients.THYME_SPRIGS: ('thyme sprigs', 'Lucky', 'weekly', 'produce')>: (6,
  <Meas.COUNT: ''>),
 <Ingredients.OLIVE_OIL: ('olive oil', 'Trader Joes', 'staple', 'ingredients')>: (8.0,
  <Meas.TBSP: 'Tablespoon'>),
 <Ingredients.GARLIC: ('garlic', 'Lucky', 'staple', 'produce')>: (18,
  <Meas.CLOVES: 'cloves'>),
 <Ingredients.HEIRLOOM_TOMATOES: ('heirloom tomatoes', 'Lucky', 'weekly', 'produce')>: (8,
  <Meas.COUNT: ''>),
 <Ingredients.WHITE_BALSAMIC: ('white balsamic vinegar', 'Lucky', 'staple', 'ingredients')>: (6,
  <Meas.TBSP: 'Tablespoon'>),
 <Ingredients

In [19]:
list_of_groceries = sl.list_of_groceries
lists = {}
for ingredient, ingredient_info in list_of_groceries.items():
  print(ingredient)
  print(type(ingredient))
  print(ingredient_info)
  print(ingredient.value)
  i = ingredient.value[0]
  store = ingredient.value[1]
  freq = ingredient.value[2]
  list_name = store + ' ' + freq
  # print(ingredient)
  # if ingredient.value[2] == 'weekly':
  if list_name in lists:
      lists[list_name].append((ingredient.value[3], i, ingredient_info[0], ingredient_info[1].value))
      print((ingredient.value[3], i, ingredient_info[0], ingredient_info[1].value))
      print('\n')
      continue
  lists[list_name] = [(ingredient.value[3], i, ingredient_info[0], ingredient_info[1].value)]
  print((ingredient.value[3], i, ingredient_info[0], ingredient_info[1].value))
  print('\n')

Ingredients.ONION_RED
<enum 'Ingredients'>
(3.0, <Meas.COUNT: ''>)
('red onion', 'Trader Joes', 'weekly', 'produce')
('produce', 'red onion', 3.0, '')


Ingredients.ONION_YELLOW
<enum 'Ingredients'>
(3.0, <Meas.COUNT: ''>)
('onion - yellow', 'Trader Joes', 'weekly', 'produce')
('produce', 'onion - yellow', 3.0, '')


Ingredients.DRY_CHICKPEAS
<enum 'Ingredients'>
(2, <Meas.LB: 'lb'>)
('dry chickpeas', 'Costco', 'staple', 'grains')
('grains', 'dry chickpeas', 2, 'lb')


Ingredients.KOSHER_SALT
<enum 'Ingredients'>
(7.5, <Meas.TSP: 'teaspoon'>)
('kosher salt', 'Lucky', 'staple', 'ingredients')
('ingredients', 'kosher salt', 7.5, 'teaspoon')


Ingredients.THYME_SPRIGS
<enum 'Ingredients'>
(6, <Meas.COUNT: ''>)
('thyme sprigs', 'Lucky', 'weekly', 'produce')
('produce', 'thyme sprigs', 6, '')


Ingredients.OLIVE_OIL
<enum 'Ingredients'>
(8.0, <Meas.TBSP: 'Tablespoon'>)
('olive oil', 'Trader Joes', 'staple', 'ingredients')
('ingredients', 'olive oil', 8.0, 'Tablespoon')


Ingredients.GARLIC


In [20]:
sl = WeeklyMeals()
sl.add_recipe(weekly_snacks, 'Mon', '1/23', 'Lunch')
sl.add_recipe(weekly_staples, 'Tues', '1/24', 'Lunch')
sl.add_recipe(stewy_chickpeas, 'Wed', '1/18', 'Lunch')
sl.add_recipe(chicken_tacos, 'Wed', '1/18', 'Dinner')
sl.add_recipe(roasted_veggies_and_sausage, 'Thurs', '1/19', 'Dinner')
sl.add_recipe(grilled_cheese_sandwich, 'Fri', '1/20', 'Dinner')
sl.add_recipe(chicken_nuggets_and_broccoli, 'Sat', '1/21', 'Lunch')
sl.add_recipe(white_bean_chili, 'Sat', '1/21', 'Dinner')
sl.add_recipe(pork_loin_and_roasted_potatoes, 'Sun', '1/22', 'Dinner')